In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_detik(category):
    all_data = []
    page = 1

    while True:
        url = f'https://www.detik.com/search/searchnews?query={category}&sortby=time&page={page}'
        req = requests.get(url)
        sop = BeautifulSoup(req.text, 'html.parser')
        articles_container = sop.find('div', class_='list media_rows list-berita')

        # Cek apakah halaman berikutnya masih tersedia
        if articles_container is None:
            break

        articles = articles_container.find_all('article')

        for article in articles:
            link = article.find('a')['href']
            date = article.find('a').find('span', class_='date').text.replace('WIB', '').replace('detikNews', '').split(',')[1]
            headline = article.find('a').find('h2').text

            ge_ = requests.get(link).text
            sop_ = BeautifulSoup(ge_, 'html.parser')
            content = sop_.find('div', class_='detail__body-text itp_bodycontent')

            # Pengecekan apakah content memiliki nilai None
            if content:
                paragraphs = content.find_all('p')
                content_ = ''.join([p.get_text(strip=True) for p in paragraphs])
            else:
                content_ = ''

            all_data.append({
                'category': category,
                'headline': headline,
                'date': date,
                'content': content_
            })

        page += 1

    return all_data

# Daftar kategori
categories = ['pemilu+2024', 'your_second_category', 'your_third_category']

# Lakukan crawling data untuk setiap kategori
all_data = []

for category in categories:
    category_data = scrape_detik(category)
    all_data.extend(category_data)

# Simpan data ke CSV
csv_file_path = 'detik_news_data.csv'
csv_columns = ['category', 'headline', 'date', 'content']

with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_columns)
    writer.writeheader()
    for data in all_data:
        writer.writerow(data)

print(f'Data telah disimpan ke {csv_file_path}')